# Моделирование капилярных эффектов и эффектов диффузии во время экстракции кофе

Процесс экстракции кофе интересен по многим причинам.

Так например, во время данного процесса кофейная частица набухает из-за того, что в ее порах задерживается вода, между молекулами кофе и воды происходит диффузия, вследствие которой из кофе "вымывается" кофеин, масло и другие вещества.

Диффузия влечет за собой важную метрику кофейного напитка - TDS (Total Dissolved Solids) - это % растворенных в напитке веществ от его общего веса.

## Smoothed particle hydrodynamics

Для моделирования движения жидкости через кофе был использован метод сглаженной гидродинамики частиц, который чаще всего использует для моделирования жидкости в наши дни.

Сила между частица представляет собой суперпозицю трех сил: сила давления между частицами, сила вязкого трения и сила поверхностного натяжения.

$$
F = F^{pressure} + F^{viscosity} + F^{tension}
$$


Так как частицы сильно удаленные от данной не вносят сильного вклада в общую силу, действующую на чатицу, то мы их просто не рассматриваем, отсюда для каждой частицы и для каждой силы есть свое ядро, описывающее определнную силу.

Ядро - это просто функция $ W(r - r_i, h) $, для которой выполнено $ \int W(r - r_i, h) \,dr = 1$.

Так, например сила давления записывается ледующим образом

$$
F^{pressure}_{i} = - \nabla p(r_{i})= -\sum_{j} m_j(\frac{p_i + p_j}{2\rho_{i}})\nabla W(r_i - r_j, h)
$$

Сила вязкого трения

$$
F^{viscosity}_{i} = \eta \sum_{j} m_j(\frac{\bar{v_i} - \bar{v_j}}{\rho_{j}})\nabla^2 W(r_i - r_j, h)
$$


Сила поверхностного натяжения

$$
F^{tension}_{i} = -\sigma \nabla^2 c_s \frac{\textbf n}{|\textbf n|},\: где  \: \textbf n = \nabla c_s
$$

В этой формуле $c_s = \sum{j} \frac{m_j}{\rho_j}W_{poly}(r_i - r_j, h)$.

В силе поверхностного натяжения используется ядро $ poly6 $, имеющее следующий вид

\begin{equation*}
W_{poly6}(r, h) =  \frac{315}{64\pi h^9}
 \begin{cases}
   (h^2 - r^2)^3 & \: \:\text{0 < r < h}\\
   0 &\text{Иначе}
 \end{cases}
\end{equation*}

Так выглядит ядро poly6 и ядро первый и второй градиенты.

![](images/poly6.png)

Ядро для сил вязкого трения и давления было взято [отсюда](https://www.researchgate.net/publication/359740383_Modeling_swelling_effects_during_coffee_extraction_with_smoothed_particle_hydrodynamics) и имеет следующий вид

\begin{equation*}
W(r, h) =  \frac{7}{478\pi h^2}
 \begin{cases}
   (3 - q)^5 - 6(2 - q)^5 + 15 (1 - q) ^ 5 &\:\text{0 $\leqslant$ q < 1}\\
   (3 - q)^5 - 6(2 - q)^5 &\text{1 $\leqslant$ q < 2}\\
   (3 - q)^5 &\text{2 $\leqslant$ q < 3}\\
   0 &\text{Иначе}
 \end{cases}
\end{equation*}


где $ q = \frac{r}{h} $

## Диффузия

Диффузия веществ описывается законом Фика. Вещества переносятся между чаcтицами вода-вода и между частица вода-кофе. Перенос вещества кофе-кофе не был рассматрен в данной модели, так как считался пренебрежимо малым. Обмен веществом между частицами кофе может осуществлятся через воду, то есть идёт "вымывание" веществ из одной частицы, а дальше "набухание" другой.

Первоначальная концентрация веществ в кофейных частицах задавалась как $ c $, в воде же концентрация этих же веществ задвалась в начальный момент времени 0.


Передача вещества описывалась законом Фика:

$$
j = -D \frac{dn}{dx}
$$

В дискретном виде:

$$
\Delta с_{12} = - \frac{jSt}{V n_0} = - \frac{3}{2} \frac{D (c_1 - c_2) \Delta t}{diameter * distance}
$$

Где $c_1, c_2$ - концентрации веществ в двух взаимодействующих молекулах, $diameter$ и $distance$ - характерный размер молекулы и расстояние между ними.
Пересчёт проводился каждую итерацию.

Также можно рассмотреть зависимость диффузии веществ в воде от температуры, приняв зависимость коэффициента диффузии от корня температуры, как для диффузии на стационарном фоне.

$$
D \sim \sqrt T
$$

## Визуализация

Можно моделировать разное количество кофеных частиц, с разными коэффициентами диффузии, с разными радиусами и тд.

Для наглядности эффекта диффузии частицы воды и кофе имеют свой собсвенный цвет в зависимости от концентрации веществ в них.

Так, например выглядит модель для 3 частиц кофе

![](images/3_grains.gif)

При этом можно следить за средней концентрацией веществ в жидкости

![](images/3_grains_conc.png)

Результаты для одной чатицы кофе

![](images/single_grain.gif)

![](images/single_grain_conc.png)

### Зависимость от температуры

Кофе, как правило завваривают с помощью кипятка (либо очень долгое время, если того требует метод заваривания). Это связано так раз-таки с тем, что при увеличении температуры концентрация "кофейных веществ" в готовом напитке увеличивается.

Это иллюстрирует график ниже

![](results/temperature_dependency.png)

## Источники

https://www.researchgate.net/publication/359740383

https://www.diva-portal.org/smash/get/diva2:573583/FULLTEXT01.pdf

https://lucasschuermann.com/writing/particle-based-fluid-simulation

